In [ ]:
# !pip install requests pandas bs4 readability-lxml langdetect PyPDF2 beautifulsoup4 requests playwright chromium



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 28.5 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 15.4 MB/s eta 0:00:00


In [ ]:
!pip -q install trafilatura==1.10.0 langdetect==1.0.9 lxml==5.3.0 pdfminer.six==20240706

In [ ]:
from google.colab import drive#


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

print(f"Per-link CSVs → {PER_LINK_DIR}")
print(f"Master CSV    → {MASTER_PATH}")

drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 19.5 MB/s eta 0:00:00
Mounted at /content/drive
Per-link CSVs → /content/drive/MyDrive/webscrape_links/links
Master CSV    → /content/drive/MyDrive/webscrape_links/master_links.csv
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Paper List for Quant Analysis 18 TOTAL LINKS
Federal (7)

https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/guide-use-generative-ai.html

https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/algorithmic-impact-assessment.html

https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/gc-ai-strategy-overview.html

https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/principles.html

https://www.canada.ca/en/treasury-board-secretariat/corporate/reports/2023-2026-data-strategy.html

https://www.tbs-sct.canada.ca/pol/doc-eng.aspx?id=32603

https://www.tbs-sct.canada.ca/pol/doc-eng.aspx?id=32592

Provincial (4)

https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_IA/Strategie_integration_IA_2021_2026.pdf

https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_cybersecurite_numerique_2024-2028/GU_bonnes_pratiques_utilisation_IA_generative_VF.pdf

https://www.publicationsduquebec.gouv.qc.ca/fileadmin/gazette/pdf_encrypte/lois_reglements/2024F/83874.pdf — only the relevant section

https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_cybersecurite_numerique_2024-2028/Strategie_cybersecurite_numerique_2024-2028.pdf

Municipal (7)

https://montrealdeclaration-responsibleai.com/

https://mtl.ged.montreal.ca/constellio/?collection=mtlca&portal=REPDOCVDM#!displayDocument/00000092887

https://donnees.montreal.ca/en/pages/our-approach

https://depot.ville.montreal.qc.ca/documents/data-governance-directive.pdf

https://mtl.ged.montreal.ca/constellio/?collection=mtlca&portal=REPDOCVDM&lg=en#!displayDocument/00000092893

https://montreal.ca/en/topics/governance-personal-information

https://montreal.ca/en/topics/confidentiality-policy

In [ ]:

# 1) Paths & Imports

from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import json
import re
import uuid
import requests
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from bs4 import BeautifulSoup
from langdetect import detect as lang_detect
import trafilatura
from pdfminer.high_level import extract_text as pdf_extract_text  # ✅ correct import
try:
    from pdfminer.high_level import extract_text as pdf_extract_text
except ImportError:
    pass

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}
TIMEOUT = 30

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)"
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) Fetchers / Scrapers
# -------------------------
def fetch_bytes(url: str, headers: Optional[Dict[str, str]]=None, timeout: int=TIMEOUT) -> bytes:
    hdrs = dict(DEFAULT_HEADERS)
    if headers:
        hdrs.update(headers)
    r = requests.get(url, headers=hdrs, timeout=timeout)
    r.raise_for_status()
    return r.content

def scrape_html(url: str,
                css_title: Optional[str]=None,
                css_text: Optional[str]=None,
                use_trafilatura: bool=True) -> Tuple[str, str]:
    """
    HTML main-content extraction:
    - Primary: trafilatura (robust boilerplate removal)
    - Fallback: CSS selection (e.g., <main>)
    """
    content = fetch_bytes(url)
    html = content.decode("utf-8", errors="ignore")

    # Primary extraction
    text = ""
    if use_trafilatura:
        text = (trafilatura.extract(html, include_comments=False, include_tables=False) or "").strip()

    # Title & optional CSS fallback
    soup = BeautifulSoup(html, "lxml")
    if css_title:
        el = soup.select_one(css_title)
        title = (el.get_text(" ", strip=True) if el else "") or (soup.title.string.strip() if soup.title and soup.title.string else "")
    else:
        title = (soup.title.string.strip() if soup.title and soup.title.string else "")

    if (not text or len(text.split()) < 50) and css_text:
        el = soup.select_one(css_text)
        if el:
            text = el.get_text("\n", strip=True)

    return title or "", text or ""

def scrape_pdf(url: str) -> Tuple[str, str]:
    data = fetch_bytes(url)
    tmp = Path("/content") / f"tmp_{uuid.uuid4().hex}.pdf"
    with tmp.open("wb") as f:
        f.write(data)
    try:
        text = pdf_extract_text(str(tmp)) or ""
    finally:
        try: tmp.unlink()
        except: pass
    filename = url.rsplit("/", 1)[-1] if "/" in url else url
    title = re.sub(r"\.pdf$", "", filename, flags=re.I)
    return title, text.strip()

# -------------------------
# 4) Normalize & Orchestrate
# -------------------------
def normalize_row(*, url: str, title: str, text: str, jurisdiction: str, source_type: str, doc_id: Optional[str]=None) -> Dict[str, Any]:
    lang = safe_lang(text)
    chars, words = count_chars_words(text)
    return {
        "doc_id": doc_id or str(uuid.uuid4()),
        "title": (title or "").strip(),
        "jurisdiction": (jurisdiction or "").strip(),
        "source_type": (source_type or "").strip(),
        "url": url,
        "retrieved_at_utc": now_utc_iso(),
        "language": lang,
        "char_count": str(chars),
        "word_count": str(words),
        "text": text,
    }

def process_link(idx: int, link_cfg: Dict[str, Any]) -> Dict[str, Any]:
    url = link_cfg["url"]
    method = link_cfg.get("method", "html").lower()

    if method == "html":
        title, text = scrape_html(
            url,
            css_title=link_cfg.get("css_title"),
            css_text=link_cfg.get("css_text"),
            use_trafilatura=link_cfg.get("use_trafilatura", True),
        )
    elif method == "pdf":
        title, text = scrape_pdf(url)
    else:
        raise ValueError(f"Unsupported method '{method}'")

    if link_cfg.get("title_override"):
        title = link_cfg["title_override"]

    row = normalize_row(
        url=url,
        title=title,
        text=text,
        jurisdiction=link_cfg.get("jurisdiction", ""),
        source_type=link_cfg.get("source_type", ""),
        doc_id=link_cfg.get("doc_id"),
    )

    warn_if_low_content(url, row["title"], int(row["char_count"]), int(row["word_count"]))

    # Save single-row CSV (custom name for this link)
    custom_path = PER_LINK_DIR / f"{link_cfg['doc_id']}.csv"
    write_single_row_csv(custom_path, row)
    print(f"[OK] Saved per-link CSV: {custom_path}")

    # Append to master
    append_to_master(MASTER_PATH, row)
    print(f"[OK] Appended to master: {MASTER_PATH}")

    return row

# -------------------------
# 5) LINK 1 CONFIG & RUN
# -------------------------
link_cfg = {
    "url": "https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/guide-use-generative-ai.html",
    "method": "html",
    "jurisdiction": "Federal",
    "source_type": "Guide",
    "doc_id": "federal_guide_use_gen_ai",
    "title_override": "Guide on the use of generative AI (Canada.ca)",
    "css_title": "main h1",   # CSS fallback title
    "css_text": "main",       # CSS fallback content (Canada.ca uses <main id="wb-cont">)
    "use_trafilatura": True,
}

# Optional: uncomment to reset master on the first link of the project
# if MASTER_PATH.exists():
#     MASTER_PATH.unlink()
#     print(f"[RESET] Removed existing master: {MASTER_PATH}")

_ = process_link(1, link_cfg)

# Quick peek at the saved row
print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "federal_guide_use_gen_ai.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/federal_guide_use_gen_ai.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                  doc_id                                         title jurisdiction source_type                                                                                                                                          url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

2. algorithmic-impact-assessment
https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/algorithmic-impact-assessment.html

In [ ]:
# =========================================
# Link 2: Canada.ca Algorithmic Impact Assessment
# File: federal_algorithmic_impact_assessment.csv
# =========================================

# -------------------------
# Config for this link
# -------------------------
link_cfg = {
    "url": "https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/algorithmic-impact-assessment.html",
    "method": "html",
    "jurisdiction": "Federal",
    "source_type": "Assessment",
    "doc_id": "federal_algorithmic_impact_assessment",
    "title_override": "Algorithmic Impact Assessment (Canada.ca)",
    "css_title": "main h1",   # CSS fallback title
    "css_text": "main",       # CSS fallback for body content
    "use_trafilatura": True,
}

# -------------------------
# Run scrape + save
# -------------------------
_ = process_link(2, link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "federal_algorithmic_impact_assessment.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/federal_algorithmic_impact_assessment.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                               doc_id                                     title jurisdiction source_type                                                                                                                                                url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

3. gc-ai-strategy-overview
https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/gc-ai-strategy-overview.html

In [ ]:
# =========================================
# Link 3: GC AI Strategy (Overview + Sections aggregator)
# File: gc_ai_strategy_overview.csv (single row)
# =========================================


import csv
import re
import uuid
import requests
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple, List
import pandas as pd
from bs4 import BeautifulSoup
from langdetect import detect as lang_detect
import trafilatura
from pdfminer.high_level import extract_text as pdf_extract_text  # not used here, but kept for parity

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}
TIMEOUT = 30

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)"
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) Fetch & Extract Helpers
# -------------------------
BASE_OVERVIEW = "https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/gc-ai-strategy-overview.html"
EXPECTED_SLUGS = [
    "gc-ai-strategy-full-text.html",
    "gc-ai-strategy-priority-areas.html",
    "gc-ai-strategy-expectations-federal-organizations.html",
    "gc-ai-strategy-relevant-strategies-policy.html",
    "gc-ai-strategy-engagement-consultation.html",
]

def fetch_bytes(url: str, headers: Optional[Dict[str, str]]=None, timeout: int=TIMEOUT) -> bytes:
    hdrs = dict(DEFAULT_HEADERS)
    if headers:
        hdrs.update(headers)
    r = requests.get(url, headers=hdrs, timeout=timeout)
    r.raise_for_status()
    return r.content

def extract_html_text(url: str,
                      css_title: Optional[str]="main h1",
                      css_text: Optional[str]="main",
                      use_trafilatura: bool=True) -> Tuple[str, str]:
    """
    Extracts title and main text from an HTML page.
    Primary: trafilatura; Fallback: CSS select of <main>.
    """
    content = fetch_bytes(url)
    html = content.decode("utf-8", errors="ignore")
    title, text = "", ""

    if use_trafilatura:
        text = (trafilatura.extract(html, include_comments=False, include_tables=False) or "").strip()

    soup = BeautifulSoup(html, "lxml")
    # Title
    if css_title:
        el = soup.select_one(css_title)
        title = (el.get_text(" ", strip=True) if el else "") or (soup.title.string.strip() if soup.title and soup.title.string else "")
    else:
        title = (soup.title.string.strip() if soup.title and soup.title.string else "")

    # Fallback body
    if (not text or len(text.split()) < 80) and css_text:
        el = soup.select_one(css_text)
        if el:
            text = el.get_text("\n", strip=True)

    return title or "", text or ""

def discover_strategy_links(overview_url: str = BASE_OVERVIEW) -> List[str]:
    """Parse the overview page and return a prioritized list of section URLs.
       Full-text first (if present), then the 4 section pages."""
    base = "https://www.canada.ca"
    html = fetch_bytes(overview_url).decode("utf-8", errors="ignore")
    soup = BeautifulSoup(html, "lxml")
    found = []
    for a in soup.select("main a[href]"):
        href = a.get("href", "").strip()
        if not href:
            continue
        # Normalize to absolute
        if href.startswith("/"):
            full = base + href
        elif href.startswith("http"):
            full = href
        else:
            # relative link
            from urllib.parse import urljoin
            full = urljoin(overview_url, href)
        # Keep only expected slugs
        if any(slug in full for slug in EXPECTED_SLUGS):
            found.append(full)

    # Deduplicate, keep order
    seen = set()
    ordered = []
    # Prefer full-text first if present
    for url in sorted(found, key=lambda u: (0 if "full-text" in u else 1, EXPECTED_SLUGS.index(u.rsplit("/",1)[-1]) if any(s in u for s in EXPECTED_SLUGS) else 99)):
        if url not in seen:
            ordered.append(url)
            seen.add(url)
    return ordered

# -------------------------
# 4) Aggregate content
# -------------------------
DOC_ID = "gc_ai_strategy_overview"
OVERVIEW_TITLE = "AI Strategy for the Federal Public Service 2025–2027 (Overview + sections)"
JURISDICTION = "Federal"
SOURCE_TYPE = "Strategy"

try:
    # Discover
    section_urls = discover_strategy_links(BASE_OVERVIEW)
    if not section_urls:
        # If nothing discovered, at least use the overview itself
        section_urls = [BASE_OVERVIEW]

    # Prefer full-text if present
    full_text_urls = [u for u in section_urls if "full-text" in u]
    fetch_list = full_text_urls if full_text_urls else section_urls

    # Fetch & build aggregate text
    parts = []
    main_title = OVERVIEW_TITLE
    for u in fetch_list:
        t, x = extract_html_text(u, css_title="main h1", css_text="main", use_trafilatura=True)
        if t and x:
            parts.append(f"### {t}\n\n{x}")
        elif x:
            parts.append(x)
        else:
            parts.append(f"[No content extracted from {u}]")

    aggregate_text = "\n\n---\n\n".join(parts).strip()
    aggregate_title = main_title

    # Normalize row
    lang = safe_lang(aggregate_text)
    chars, words = count_chars_words(aggregate_text)
    row = {
        "doc_id": DOC_ID,
        "title": aggregate_title,
        "jurisdiction": JURISDICTION,
        "source_type": SOURCE_TYPE,
        "url": BASE_OVERVIEW,
        "retrieved_at_utc": now_utc_iso(),
        "language": lang,
        "char_count": str(chars),
        "word_count": str(words),
        "text": aggregate_text,
    }

    warn_if_low_content(BASE_OVERVIEW, aggregate_title, chars, words)

    # Save single-row CSV
    out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
    write_single_row_csv(out_path, row)
    print(f"[OK] Saved: {out_path}")

    # Append to master
    append_to_master(MASTER_PATH, row)
    print(f"[OK] Appended to master: {MASTER_PATH}")

    # Preview
    print("\nPreview:")
    print(pd.read_csv(out_path, dtype=str).head(1).to_string(index=False))

    # Show which URLs were used
    print("\nIncluded URLs:")
    for u in fetch_list:
        print(" -", u)

except Exception as e:
    # On error, still write a row to keep 1-row-per-link invariant
    err_text = f"{type(e).__name__}: {e}"
    row = {
        "doc_id": DOC_ID,
        "title": "[ERROR] GC AI Strategy aggregation",
        "jurisdiction": JURISDICTION,
        "source_type": SOURCE_TYPE,
        "url": BASE_OVERVIEW,
        "retrieved_at_utc": now_utc_iso(),
        "language": "",
        "char_count": str(len(err_text)),
        "word_count": str(len(err_text.split())),
        "text": err_text,
    }
    warn_if_low_content(BASE_OVERVIEW, row["title"], int(row["char_count"]), int(row["word_count"]))
    out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
    write_single_row_csv(out_path, row)
    append_to_master(MASTER_PATH, row)
    print(f"[ERROR] {e}\n[OK] Wrote error row and appended to master.")


[OK] Saved: /content/drive/MyDrive/webscrape_links/links/gc_ai_strategy_overview.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                 doc_id                                                                      title jurisdiction source_type                                                                                                                                          url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

4. federal_responsible_use_ai
https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/principles.html

In [ ]:
# =========================================
# Link 4: Guiding principles for the use of AI in government (Canada.ca)
# File: federal_responsible_use_ai.csv
# =========================================

# Assumes you've already run the initial setup cells that define:
# - BASE_DIR, PER_LINK_DIR, MASTER_PATH
# - SCHEMA, warn_if_low_content, ensure_schema, write_single_row_csv, append_to_master
# - fetch_bytes, scrape_html/scrape_pdf (we'll use scrape_html)
# - normalize_row, process_link

link_cfg = {
    "url": "https://www.canada.ca/en/government/system/digital-government/digital-government-innovations/responsible-use-ai/principles.html",
    "method": "html",
    "jurisdiction": "Federal",
    "source_type": "Principles",
    "doc_id": "federal_responsible_use_ai",
    "title_override": "Guiding principles for the use of AI in government (Canada.ca)",
    # Robust extraction + CSS fallback to <main> region if auto-extraction is thin
    "css_title": "main h1",
    "css_text": "main",
    "use_trafilatura": True,
}

_ = process_link(4, link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "federal_responsible_use_ai.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/federal_responsible_use_ai.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                    doc_id                                                          title jurisdiction source_type                                                                                                                             url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

5. federal_2023_26_data_strategy
https://www.canada.ca/en/treasury-board-secretariat/corporate/reports/2023-2026-data-strategy.html

In [ ]:
# =========================================
# Link 5: 2023–2026 Data Strategy (TBS)
# File: federal_2023_26_data_strategy.csv
# =========================================

# Assumes you've already run the setup cells (install/mount/helpers/process_link)

link_cfg = {
    "url": "https://www.canada.ca/en/treasury-board-secretariat/corporate/reports/2023-2026-data-strategy.html",
    "method": "html",
    "jurisdiction": "Federal",
    "source_type": "Strategy",
    "doc_id": "federal_2023_26_data_strategy",
    "title_override": "2023–2026 Data Strategy for the Federal Public Service (TBS)",
    # Robust extraction + CSS fallback to <main> region if auto-extraction is thin
    "css_title": "main h1",
    "css_text": "main",
    "use_trafilatura": True,
}

_ = process_link(5, link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "federal_2023_26_data_strategy.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/federal_2023_26_data_strategy.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                       doc_id                                                        title jurisdiction source_type                                                                                                url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

6. policy_service_digital
https://www.tbs-sct.canada.ca/pol/doc-eng.aspx?id=32603

In [ ]:
# =========================================
# Link 6: Policy on Service and Digital (TBS)
# File: policy_service_digital.csv
# =========================================

# Assumes you already ran the setup cells that define:
# - BASE_DIR, PER_LINK_DIR, MASTER_PATH
# - SCHEMA utils: warn_if_low_content, ensure_schema, write_single_row_csv, append_to_master
# - fetch_bytes, scrape_html (HTML), scrape_pdf (unused), normalize_row, process_link

link_cfg = {
    "url": "https://www.tbs-sct.canada.ca/pol/doc-eng.aspx?id=32603",
    "method": "html",
    "jurisdiction": "Federal",
    "source_type": "Policy",
    "doc_id": "policy_service_digital",
    "title_override": "Policy on Service and Digital (TBS)",
    # TBS pages sometimes use different wrappers; select from several common containers:
    # select_one() will return the first that exists.
    "css_title": "main h1, #wb-main h1, #wb-cont h1, #cont h1, h1",
    "css_text":  "main, #wb-main, #wb-cont, #cont, #content, .content",
    "use_trafilatura": True,
}

_ = process_link(6, link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "policy_service_digital.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/policy_service_digital.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                doc_id                               title jurisdiction source_type                                                     url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

7. federal_automated_decision_making

https://www.tbs-sct.canada.ca/pol/doc-eng.aspx?id=32592

In [ ]:
# =========================================
# Link 7: Directive on Automated Decision-Making (TBS)
# File: federal_automated_decision_making.csv
# =========================================

link_cfg = {
    "url": "https://www.tbs-sct.canada.ca/pol/doc-eng.aspx?id=32592",
    "method": "html",
    "jurisdiction": "Federal",
    "source_type": "Directive",
    "doc_id": "federal_automated_decision_making",
    "title_override": "Directive on Automated Decision-Making (TBS)",
    # TBS pages sometimes vary in structure; include multiple fallbacks:
    # - Title: try main h1, then wb-main/wb-cont, then any h1.
    # - Text:  try main, wb-main, wb-cont, cont, content, or generic .content container.
    "css_title": "main h1, #wb-main h1, #wb-cont h1, #cont h1, h1",
    "css_text":  "main, #wb-main, #wb-cont, #cont, #content, .content",
    "use_trafilatura": True,
}

_ = process_link(7, link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "federal_automated_decision_making.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/federal_automated_decision_making.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                           doc_id                                        title jurisdiction source_type                                                     url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

PROVINCIAL

Link 1. provincial_strategie_integration_IA

https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_IA/Strategie_integration_IA_2021_2026.pdf

In [ ]:
# =========================================
# Provincial Link 1 (Québec)
# Title: Stratégie d’intégration de l’IA 2021–2026
# File: provincial_strategie_integration_IA.csv
# Method: PDF -> pdfminer.six text extraction
# =========================================

# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
import requests
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}
TIMEOUT = 45

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only PDF, consider adding OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) PDF Fetch & Extract
# -------------------------
def fetch_bytes(url: str, headers: Optional[Dict[str, str]]=None, timeout: int=TIMEOUT) -> bytes:
    hdrs = dict(DEFAULT_HEADERS)
    if headers:
        hdrs.update(headers)
    r = requests.get(url, headers=hdrs, timeout=timeout)
    r.raise_for_status()
    return r.content

def scrape_pdf(url: str) -> Tuple[str, str]:
    """
    Download the PDF and extract text with pdfminer.six.
    Title is set via override below; filename fallback is kept for reference.
    """
    data = fetch_bytes(url)
    tmp_pdf = Path("/content") / f"tmp_{uuid.uuid4().hex}.pdf"
    with tmp_pdf.open("wb") as f:
        f.write(data)
    try:
        text = pdf_extract_text(str(tmp_pdf)) or ""
    finally:
        try: tmp_pdf.unlink()
        except: pass
    filename = url.rsplit("/", 1)[-1] if "/" in url else url
    title = re.sub(r"\.pdf$", "", filename, flags=re.I)
    return title, text.strip()

def normalize_row(*, url: str, title: str, text: str, jurisdiction: str, source_type: str, doc_id: Optional[str]=None) -> Dict[str, Any]:
    lang = safe_lang(text)
    chars, words = count_chars_words(text)
    return {
        "doc_id": doc_id or str(uuid.uuid4()),
        "title": (title or "").strip(),
        "jurisdiction": (jurisdiction or "").strip(),
        "source_type": (source_type or "").strip(),
        "url": url,
        "retrieved_at_utc": now_utc_iso(),
        "language": lang,
        "char_count": str(chars),
        "word_count": str(words),
        "text": text,
    }

def process_pdf_link(link_cfg: Dict[str, Any]) -> Dict[str, Any]:
    url = link_cfg["url"]
    title, text = scrape_pdf(url)
    if link_cfg.get("title_override"):
        title = link_cfg["title_override"]

    row = normalize_row(
        url=url,
        title=title,
        text=text,
        jurisdiction=link_cfg.get("jurisdiction", ""),
        source_type=link_cfg.get("source_type", ""),
        doc_id=link_cfg.get("doc_id"),
    )

    warn_if_low_content(url, row["title"], int(row["char_count"]), int(row["word_count"]))

    out_path = PER_LINK_DIR / f"{link_cfg['doc_id']}.csv"
    write_single_row_csv(out_path, row)
    print(f"[OK] Saved per-link CSV: {out_path}")

    append_to_master(MASTER_PATH, row)
    print(f"[OK] Appended to master: {MASTER_PATH}")

    return row

# -------------------------
# 4) LINK CONFIG & RUN
# -------------------------
link_cfg = {
    "url": "https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_IA/Strategie_integration_IA_2021_2026.pdf",
    "method": "pdf",
    "jurisdiction": "Provincial-Quebec",
    "source_type": "Strategy",
    "doc_id": "provincial_strategie_integration_IA",
    "title_override": "Stratégie d’intégration de l’IA 2021–2026 (Gouvernement du Québec)",
}

_ = process_pdf_link(link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "provincial_strategie_integration_IA.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/provincial_strategie_integration_IA.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                             doc_id                                                              title      jurisdiction source_type                                                                                                                                        url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

2. provincial_good_practices_gen_AI

https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_cybersecurite_numerique_2024-2028/GU_bonnes_pratiques_utilisation_IA_generative_VF.pdf

In [ ]:
# =========================================
# Provincial Link 2 (Québec)
# Title: Bonnes pratiques – utilisation de l’IA générative
# File: provincial_good_practices_gen_AI.csv
# Method: PDF -> pdfminer.six text extraction
# =========================================

# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
import requests
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}
TIMEOUT = 45

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only PDF, consider adding OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) PDF Fetch & Extract
# -------------------------
def fetch_bytes(url: str, headers: Optional[Dict[str, str]]=None, timeout: int=TIMEOUT) -> bytes:
    hdrs = dict(DEFAULT_HEADERS)
    if headers:
        hdrs.update(headers)
    r = requests.get(url, headers=hdrs, timeout=timeout)
    r.raise_for_status()
    return r.content

def scrape_pdf(url: str) -> Tuple[str, str]:
    """
    Download the PDF and extract text with pdfminer.six.
    Title is set via override below; filename fallback is kept for reference.
    """
    data = fetch_bytes(url)
    tmp_pdf = Path("/content") / f"tmp_{uuid.uuid4().hex}.pdf"
    with tmp_pdf.open("wb") as f:
        f.write(data)
    try:
        text = pdf_extract_text(str(tmp_pdf)) or ""
    finally:
        try: tmp_pdf.unlink()
        except: pass
    filename = url.rsplit("/", 1)[-1] if "/" in url else url
    title = re.sub(r"\.pdf$", "", filename, flags=re.I)
    return title, text.strip()

def normalize_row(*, url: str, title: str, text: str, jurisdiction: str, source_type: str, doc_id: Optional[str]=None) -> Dict[str, Any]:
    lang = safe_lang(text)
    chars, words = count_chars_words(text)
    return {
        "doc_id": doc_id or str(uuid.uuid4()),
        "title": (title or "").strip(),
        "jurisdiction": (jurisdiction or "").strip(),
        "source_type": (source_type or "").strip(),
        "url": url,
        "retrieved_at_utc": now_utc_iso(),
        "language": lang,
        "char_count": str(chars),
        "word_count": str(words),
        "text": text,
    }

def process_pdf_link(link_cfg: Dict[str, Any]) -> Dict[str, Any]:
    url = link_cfg["url"]
    title, text = scrape_pdf(url)
    if link_cfg.get("title_override"):
        title = link_cfg["title_override"]

    row = normalize_row(
        url=url,
        title=title,
        text=text,
        jurisdiction=link_cfg.get("jurisdiction", ""),
        source_type=link_cfg.get("source_type", ""),
        doc_id=link_cfg.get("doc_id"),
    )

    warn_if_low_content(url, row["title"], int(row["char_count"]), int(row["word_count"]))

    out_path = PER_LINK_DIR / f"{link_cfg['doc_id']}.csv"
    write_single_row_csv(out_path, row)
    print(f"[OK] Saved per-link CSV: {out_path}")

    append_to_master(MASTER_PATH, row)
    print(f"[OK] Appended to master: {MASTER_PATH}")

    return row

# -------------------------
# 4) LINK CONFIG & RUN
# -------------------------
link_cfg = {
    "url": "https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_cybersecurite_numerique_2024-2028/GU_bonnes_pratiques_utilisation_IA_generative_VF.pdf",
    "method": "pdf",
    "jurisdiction": "Provincial-Quebec",
    "source_type": "Guidance",
    "doc_id": "provincial_good_practices_gen_AI",
    "title_override": "Bonnes pratiques pour l’utilisation de l’IA générative (Gouvernement du Québec)",
}

_ = process_pdf_link(link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "provincial_good_practices_gen_AI.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/provincial_good_practices_gen_AI.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                          doc_id                                                                           title      jurisdiction source_type                                                                                                                                                                                     url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                   

3. provincial_responsible_public_AI
https://www.publicationsduquebec.gouv.qc.ca/fileadmin/gazette/pdf_encrypte/lois_reglements/2024F/83874.pdf <-- relevant sections

In [ ]:
# =========================================
# Provincial Link 3 (Québec)
# File: provincial_responsible_public_AI.csv
# URL:  https://www.publicationsduquebec.gouv.qc.ca/fileadmin/gazette/pdf_encrypte/lois_reglements/2024F/83874.pdf
# Method: PDF -> pdfminer.six (primary) with PyPDF fallback for encrypted PDFs
# =========================================



# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
import requests
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text
from pypdf import PdfReader

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}
TIMEOUT = 60

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only or strongly encrypted PDF, we can add OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) PDF Fetch & Robust Extract
# -------------------------
def fetch_bytes(url: str, headers: Optional[Dict[str, str]]=None, timeout: int=TIMEOUT) -> bytes:
    hdrs = dict(DEFAULT_HEADERS)
    if headers:
        hdrs.update(headers)
    r = requests.get(url, headers=hdrs, timeout=timeout)
    r.raise_for_status()
    return r.content

def extract_with_pdfminer(pdf_path: Path) -> str:
    try:
        return pdf_extract_text(str(pdf_path)) or ""
    except Exception as e:
        # pdfminer may fail on some encrypted PDFs
        return ""

def extract_with_pypdf(pdf_path: Path) -> str:
    try:
        reader = PdfReader(str(pdf_path))
        # Try decrypt with empty password if needed
        if reader.is_encrypted:
            try:
                reader.decrypt("")  # empty password often works for public docs
            except Exception:
                pass
        text_parts = []
        for page in reader.pages:
            try:
                text_parts.append(page.extract_text() or "")
            except Exception:
                text_parts.append("")
        return "\n".join(text_parts).strip()
    except Exception:
        return ""

def scrape_pdf_robust(url: str) -> Tuple[str, str]:
    data = fetch_bytes(url)
    tmp_pdf = Path("/content") / f"tmp_{uuid.uuid4().hex}.pdf"
    with tmp_pdf.open("wb") as f:
        f.write(data)
    try:
        # 1) Try pdfminer (best for layout-aware text)
        text = extract_with_pdfminer(tmp_pdf)

        # 2) If too thin, fall back to PyPDF (handles many encrypted PDFs)
        if len(text.split()) < 50:
            fallback = extract_with_pypdf(tmp_pdf)
            if len(fallback.split()) > len(text.split()):
                text = fallback

    finally:
        try: tmp_pdf.unlink()
        except: pass

    filename = url.rsplit("/", 1)[-1] if "/" in url else url
    title = re.sub(r"\.pdf$", "", filename, flags=re.I)
    return title, (text or "").strip()

def normalize_row(*, url: str, title: str, text: str, jurisdiction: str, source_type: str, doc_id: Optional[str]=None) -> Dict[str, Any]:
    lang = safe_lang(text)
    chars, words = count_chars_words(text)
    return {
        "doc_id": doc_id or str(uuid.uuid4()),
        "title": (title or "").strip(),
        "jurisdiction": (jurisdiction or "").strip(),
        "source_type": (source_type or "").strip(),
        "url": url,
        "retrieved_at_utc": now_utc_iso(),
        "language": lang,
        "char_count": str(chars),
        "word_count": str(words),
        "text": text,
    }

def process_pdf_link(link_cfg: Dict[str, Any]) -> Dict[str, Any]:
    url = link_cfg["url"]
    title, text = scrape_pdf_robust(url)

    if link_cfg.get("title_override"):
        title = link_cfg["title_override"]

    row = normalize_row(
        url=url,
        title=title,
        text=text,
        jurisdiction=link_cfg.get("jurisdiction", ""),
        source_type=link_cfg.get("source_type", ""),
        doc_id=link_cfg.get("doc_id"),
    )

    warn_if_low_content(url, row["title"], int(row["char_count"]), int(row["word_count"]))

    out_path = PER_LINK_DIR / f"{link_cfg['doc_id']}.csv"
    write_single_row_csv(out_path, row)
    print(f"[OK] Saved per-link CSV: {out_path}")

    append_to_master(MASTER_PATH, row)
    print(f"[OK] Appended to master: {MASTER_PATH}")

    return row

# -------------------------
# 4) LINK CONFIG & RUN
# -------------------------
link_cfg = {
    "url": "https://www.publicationsduquebec.gouv.qc.ca/fileadmin/gazette/pdf_encrypte/lois_reglements/2024F/83874.pdf",
    "method": "pdf",
    "jurisdiction": "Provincial-Quebec",
    "source_type": "Regulation",  # Gazette des lois et règlements
    "doc_id": "provincial_responsible_public_AI",
    "title_override": "Gazette officielle du Québec – Document 83874 (2024F)",
}

_ = process_pdf_link(link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "provincial_responsible_public_AI.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/provincial_responsible_public_AI.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                          doc_id                                                 title      jurisdiction source_type                                                                                                        url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

4. provincial_strat_cybersecurity_2024-28
https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_cybersecurite_numerique_2024-2028/Strategie_cybersecurite_numerique_2024-2028.pdf

In [ ]:
# =========================================
# Provincial Link 4 (Québec)
# Title: Stratégie de cybersécurité numérique 2024–2028
# File: provincial_strat_cybersecurity_2024-28.csv
# Method: PDF -> pdfminer.six (primary) + pypdf fallback
# =========================================


# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
import requests
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text
from pypdf import PdfReader

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}
TIMEOUT = 60

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only PDF, we can add OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) PDF Fetch & Robust Extract
# -------------------------
def fetch_bytes(url: str, headers: Optional[Dict[str, str]]=None, timeout: int=TIMEOUT) -> bytes:
    hdrs = dict(DEFAULT_HEADERS)
    if headers:
        hdrs.update(headers)
    r = requests.get(url, headers=hdrs, timeout=timeout)
    r.raise_for_status()
    return r.content

def extract_with_pdfminer(pdf_path: Path) -> str:
    try:
        return pdf_extract_text(str(pdf_path)) or ""
    except Exception:
        return ""

def extract_with_pypdf(pdf_path: Path) -> str:
    try:
        reader = PdfReader(str(pdf_path))
        if reader.is_encrypted:
            try:
                reader.decrypt("")  # try empty password (often works for public docs)
            except Exception:
                pass
        out = []
        for page in reader.pages:
            try:
                out.append(page.extract_text() or "")
            except Exception:
                out.append("")
        return "\n".join(out).strip()
    except Exception:
        return ""

def scrape_pdf_robust(url: str) -> Tuple[str, str]:
    data = fetch_bytes(url)
    tmp_pdf = Path("/content") / f"tmp_{uuid.uuid4().hex}.pdf"
    with tmp_pdf.open("wb") as f:
        f.write(data)
    try:
        text = extract_with_pdfminer(tmp_pdf)
        if len(text.split()) < 50:  # thin? fall back
            alt = extract_with_pypdf(tmp_pdf)
            if len(alt.split()) > len(text.split()):
                text = alt
    finally:
        try: tmp_pdf.unlink()
        except: pass

    filename = url.rsplit("/", 1)[-1] if "/" in url else url
    title = re.sub(r"\.pdf$", "", filename, flags=re.I)
    return title, (text or "").strip()

# -------------------------
# 4) Normalize & Save
# -------------------------
def normalize_row(*, url: str, title: str, text: str, jurisdiction: str, source_type: str, doc_id: Optional[str]=None) -> Dict[str, Any]:
    lang = safe_lang(text)
    chars, words = count_chars_words(text)
    return {
        "doc_id": doc_id or str(uuid.uuid4()),
        "title": (title or "").strip(),
        "jurisdiction": (jurisdiction or "").strip(),
        "source_type": (source_type or "").strip(),
        "url": url,
        "retrieved_at_utc": now_utc_iso(),
        "language": lang,
        "char_count": str(chars),
        "word_count": str(words),
        "text": text,
    }

def process_pdf_link(link_cfg: Dict[str, Any]) -> Dict[str, Any]:
    url = link_cfg["url"]
    title, text = scrape_pdf_robust(url)
    if link_cfg.get("title_override"):
        title = link_cfg["title_override"]

    row = normalize_row(
        url=url,
        title=title,
        text=text,
        jurisdiction=link_cfg.get("jurisdiction", ""),
        source_type=link_cfg.get("source_type", ""),
        doc_id=link_cfg.get("doc_id"),
    )

    warn_if_low_content(url, row["title"], int(row["char_count"]), int(row["word_count"]))

    out_path = PER_LINK_DIR / f"{link_cfg['doc_id']}.csv"
    write_single_row_csv(out_path, row)
    print(f"[OK] Saved per-link CSV: {out_path}")

    append_to_master(MASTER_PATH, row)
    print(f"[OK] Appended to master: {MASTER_PATH}")
    return row

# -------------------------
# 5) LINK CONFIG & RUN
# -------------------------
link_cfg = {
    "url": "https://cdn-contenu.quebec.ca/cdn-contenu/adm/min/cybersecurite_numerique/Publications/Strategie_cybersecurite_numerique_2024-2028/Strategie_cybersecurite_numerique_2024-2028.pdf",
    "method": "pdf",
    "jurisdiction": "Provincial-Quebec",
    "source_type": "Strategy",
    "doc_id": "provincial_strat_cybersecurity_2024-28",
    "title_override": "Stratégie de cybersécurité numérique 2024–2028 (Gouvernement du Québec)",
}

_ = process_pdf_link(link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "provincial_strat_cybersecurity_2024-28.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/provincial_strat_cybersecurity_2024-28.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                                doc_id                                                                   title      jurisdiction source_type                                                                                                                                                                                url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                    

MUNICIPALITY - MONTREAL

1. municipality_responsible_AI
https://montrealdeclaration-responsibleai.com/

In [ ]:
# =========================================
# Municipality Link 1: Montreal Declaration for Responsible AI
# Domain: https://montrealdeclaration-responsibleai.com/
# File: municipality_responsible_AI.csv
# Method: HTML hub -> discover internal content pages -> aggregate into one row
# =========================================

# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import time
import uuid
import requests
from urllib.parse import urljoin, urlparse
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple, List, Set

import pandas as pd
from bs4 import BeautifulSoup
from langdetect import detect as lang_detect
import trafilatura

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}
TIMEOUT = 30

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)"
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) Fetch & Extract Helpers
# -------------------------
START_URL = "https://montrealdeclaration-responsibleai.com/"
ALLOWED_NETLOC = urlparse(START_URL).netloc

INCLUDE_PATTERNS = [
    r"/(declaration|principles?|about|objectives|commitments|eng|en|anglais|english)/?",
    r"/(la-declaration|principes|a-propos|engagements|fr)/?",
]
EXCLUDE_PATTERNS = [
    r"/(wp-json|feed|xmlrpc|tag|category|author)/",
    r"\.(pdf|jpg|jpeg|png|gif|svg|zip|mp4|mp3|avi|mov)(\?|$)",
    r"#",
]

def fetch_bytes(url: str, headers: Optional[Dict[str, str]]=None, timeout: int=TIMEOUT) -> bytes:
    hdrs = dict(DEFAULT_HEADERS)
    if headers:
        hdrs.update(headers)
    r = requests.get(url, headers=hdrs, timeout=timeout)
    r.raise_for_status()
    return r.content

def should_consider(url: str) -> bool:
    p = urlparse(url)
    if p.netloc and p.netloc != ALLOWED_NETLOC:
        return False
    for pat in EXCLUDE_PATTERNS:
        if re.search(pat, url, flags=re.I):
            return False
    if any(re.search(pat, url, flags=re.I) for pat in INCLUDE_PATTERNS):
        return True
    # Allow top-level pages (/, /en/, /fr/) even if no pattern matched
    if p.path in ("", "/", "/en/", "/fr/"):
        return True
    return False

def discover_internal_links(start_url: str = START_URL, max_pages: int = 12) -> List[str]:
    """Crawl homepage, collect internal links (depth ≤ 1), filter to likely content pages."""
    try:
        html = fetch_bytes(start_url).decode("utf-8", errors="ignore")
    except Exception as e:
        print(f"[WARN] Failed to fetch start URL: {e}")
        return [start_url]

    soup = BeautifulSoup(html, "lxml")
    links: List[str] = []
    for a in soup.select("a[href]"):
        href = a.get("href", "").strip()
        if not href:
            continue
        full = urljoin(start_url, href)
        if should_consider(full):
            links.append(full)

    # Deduplicate & limit
    seen: Set[str] = set()
    ordered: List[str] = []
    for u in links:
        if u not in seen:
            ordered.append(u)
            seen.add(u)

    # Prioritize English pages if both EN/FR exist
    def lang_priority(u: str) -> int:
        # Prefer English paths if available; otherwise neutral.
        if "/en" in u or "english" in u:
            return 0
        return 1

    ordered = sorted(ordered, key=lang_priority)[:max_pages]

    # Ensure the start URL is included at least once
    if start_url not in ordered:
        ordered.insert(0, start_url)

    return ordered

def extract_html_text(url: str,
                      css_title: Optional[str]="main h1, header h1, .entry-title, h1",
                      css_text: Optional[str]="main, article, .entry-content, .post-content, .content, #content",
                      use_trafilatura: bool=True) -> Tuple[str, str]:
    """Return (title, text) from an HTML page."""
    content = fetch_bytes(url)
    html = content.decode("utf-8", errors="ignore")
    title, text = "", ""

    if use_trafilatura:
        text = (trafilatura.extract(html, include_comments=False, include_tables=False) or "").strip()

    soup = BeautifulSoup(html, "lxml")

    # Title
    if css_title:
        el = soup.select_one(css_title)
        title = (el.get_text(" ", strip=True) if el else "") or (soup.title.string.strip() if soup.title and soup.title.string else "")
    else:
        title = (soup.title.string.strip() if soup.title and soup.title.string else "")

    # Fallback body if thin
    if (not text or len(text.split()) < 80) and css_text:
        el = soup.select_one(css_text)
        if el:
            text = el.get_text("\n", strip=True)

    return title or "", text or ""

# -------------------------
# 4) Aggregate content from discovered pages
# -------------------------
DOC_ID = "municipality_responsible_AI"
CANONICAL_TITLE = "Montreal Declaration for a Responsible Development of Artificial Intelligence"
JURISDICTION = "Municipal-Montreal"
SOURCE_TYPE = "Declaration"

try:
    urls = discover_internal_links(START_URL, max_pages=12)

    parts = []
    titles_seen = set()
    for u in urls:
        try:
            t, x = extract_html_text(u, use_trafilatura=True)
        except Exception as e:
            t, x = "", f"[Fetch error for {u}: {type(e).__name__} {e}]"

        # Skip truly empty pages
        if not (t or x):
            continue

        # Deduplicate near-duplicate section titles to keep output concise
        norm_t = (t or "").strip()
        if norm_t and norm_t.lower() in titles_seen:
            section_heading = norm_t
        else:
            section_heading = norm_t
            if norm_t:
                titles_seen.add(norm_t.lower())

        # Append with a visible section break
        if section_heading and x:
            parts.append(f"### {section_heading}\n\n{x}")
        elif x:
            parts.append(x)
        else:
            parts.append(section_heading)

    aggregate_text = "\n\n---\n\n".join(parts).strip()

    # Build row
    lang = safe_lang(aggregate_text)
    chars, words = count_chars_words(aggregate_text)
    row = {
        "doc_id": DOC_ID,
        "title": CANONICAL_TITLE,
        "jurisdiction": JURISDICTION,
        "source_type": SOURCE_TYPE,
        "url": START_URL,
        "retrieved_at_utc": now_utc_iso(),
        "language": lang,
        "char_count": str(chars),
        "word_count": str(words),
        "text": aggregate_text,
    }

    warn_if_low_content(START_URL, row["title"], chars, words)

    out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
    write_single_row_csv(out_path, row)
    print(f"[OK] Saved per-link CSV: {out_path}")

    append_to_master(MASTER_PATH, row)
    print(f"[OK] Appended to master: {MASTER_PATH}")

    # Preview & traceability
    print("\nPreview:")
    print(pd.read_csv(out_path, dtype=str).head(1).to_string(index=False))

    print("\nIncluded URLs:")
    for u in urls:
        print(" -", u)

except Exception as e:
    err_text = f"{type(e).__name__}: {e}"
    row = {
        "doc_id": DOC_ID,
        "title": "[ERROR] Montreal Declaration aggregation",
        "jurisdiction": JURISDICTION,
        "source_type": SOURCE_TYPE,
        "url": START_URL,
        "retrieved_at_utc": now_utc_iso(),
        "language": "",
        "char_count": str(len(err_text)),
        "word_count": str(len(err_text.split())),
        "text": err_text,
    }
    warn_if_low_content(START_URL, row["title"], int(row["char_count"]), int(row["word_count"]))
    out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
    write_single_row_csv(out_path, row)
    append_to_master(MASTER_PATH, row)
    print(f"[ERROR] {e}\n[OK] Wrote error row and appended to master.")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/municipality_responsible_AI.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                     doc_id                                                                         title       jurisdiction source_type                                            url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

2. municipality_strat_responsible_AI
https://mtl.ged.montreal.ca/constellio/?collection=mtlca&portal=REPDOCVDM#!displayDocument/00000092887

In [ ]:
# =========================================
# Municipality Link 2 (uploaded PDF)
# File (CSV out): municipality_strat_responsible_AI.csv
# Strict schema + low-content warning
# =========================================

# -------------------------
# 0) Install & Mount Drive
# -------------------------
!pip -q install langdetect==1.0.9 pdfminer.six==20240706 pypdf==4.2.0

from google.colab import drive, files  # ✅ include files here
drive.mount('/content/drive', force_remount=True)

# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text
from pypdf import PdfReader

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only PDF, we can add OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) Robust PDF Text Extraction (from uploaded file)
# -------------------------
def extract_with_pdfminer(pdf_path: Path) -> str:
    try:
        return pdf_extract_text(str(pdf_path)) or ""
    except Exception:
        return ""

def extract_with_pypdf(pdf_path: Path) -> Tuple[str, str]:
    """Return (metadata_title, text) using pypdf (handles many encrypted PDFs)."""
    try:
        reader = PdfReader(str(pdf_path))
        if reader.is_encrypted:
            try:
                reader.decrypt("")  # empty password often works for public docs
            except Exception:
                pass
        meta_title = ""
        try:
            info = reader.metadata
            if info and getattr(info, "title", None):
                meta_title = str(info.title)
        except Exception:
            meta_title = ""
        out = []
        for page in reader.pages:
            try:
                out.append(page.extract_text() or "")
            except Exception:
                out.append("")
        return (meta_title or "").strip(), "\n".join(out).strip()
    except Exception:
        return "", ""

def scrape_pdf_local(pdf_path: Path) -> Tuple[str, str]:
    """Try pdfminer first; if thin, fall back to pypdf. Prefer metadata title."""
    text = extract_with_pdfminer(pdf_path)
    meta_title, alt_text = extract_with_pypdf(pdf_path)
    chosen_text = alt_text if len(alt_text.split()) > len(text.split()) else text
    title = meta_title or pdf_path.stem
    return title, (chosen_text or "").strip()

# -------------------------
# 4) Upload the PDF from your computer
# -------------------------
print("Please choose the local PDF to upload…")
uploaded = files.upload()  # ✅ works now
if not uploaded:
    raise RuntimeError("No file uploaded.")

uploaded_name = next(iter(uploaded.keys()))
src_path = Path("/content") / uploaded_name
pdf_path = Path("/content") / f"upload_{uuid.uuid4().hex}.pdf"
src_path.rename(pdf_path)
print(f"[OK] Uploaded & stored temporarily at: {pdf_path}")

# -------------------------
# 5) Normalize & Save to schema
# -------------------------
DOC_ID = "municipality_strat_responsible_AI"
JURISDICTION = "Municipal-Montreal"
SOURCE_TYPE = "Strategy"
SOURCE_URL = "local-upload"  # uploaded file

title, text = scrape_pdf_local(pdf_path)

row = {
    "doc_id": DOC_ID,
    "title": title,
    "jurisdiction": JURISDICTION,
    "source_type": SOURCE_TYPE,
    "url": SOURCE_URL,
    "retrieved_at_utc": now_utc_iso(),
    "language": safe_lang(text),
    "char_count": str(len(text)),
    "word_count": str(len(text.split())),
    "text": text,
}

warn_if_low_content(SOURCE_URL, row["title"], int(row["char_count"]), int(row["word_count"]))

out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
write_single_row_csv(out_path, row)
print(f"[OK] Saved per-link CSV: {out_path}")

append_to_master(MASTER_PATH, row)
print(f"[OK] Appended to master: {MASTER_PATH}")

# Cleanup temp file
try:
    pdf_path.unlink()
except Exception:
    pass

print("\nPreview:")
print(pd.read_csv(out_path, dtype=str).head(1).to_string(index=False))


Mounted at /content/drive
Please choose the local PDF to upload…


Saving montreal_ged_92887_Stratégie intelligence artificielle 2024_Ville de Montréal.pdf to montreal_ged_92887_Stratégie intelligence artificielle 2024_Ville de Montréal.pdf
[OK] Uploaded & stored temporarily at: /content/upload_e12a6b70d5274063bf2b739f14a9519f.pdf
[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/municipality_strat_responsible_AI.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                           doc_id                                   title       jurisdiction source_type          url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                     

3.  municipality_digital_data_charter
 https://mtl.ged.montreal.ca/constellio/?collection=mtlca&portal=REPDOCVDM&lg=en#!displayDocument/00000092893

In [ ]:
# =========================================
# Municipality Link (uploaded PDF)
# File (CSV out): municipality_digital_data_charter.csv
# Strict schema + low-content warning
# =========================================

# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text
from pypdf import PdfReader

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only PDF, we can add OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) Robust PDF Text Extraction (from uploaded file)
# -------------------------
def extract_with_pdfminer(pdf_path: Path) -> str:
    try:
        return pdf_extract_text(str(pdf_path)) or ""
    except Exception:
        return ""

def extract_with_pypdf(pdf_path: Path) -> Tuple[str, str]:
    """Return (metadata_title, text) using pypdf (handles many encrypted PDFs)."""
    try:
        reader = PdfReader(str(pdf_path))
        if reader.is_encrypted:
            try:
                reader.decrypt("")  # empty password often works for public docs
            except Exception:
                pass
        meta_title = ""
        try:
            info = reader.metadata
            if info and getattr(info, "title", None):
                meta_title = str(info.title)
        except Exception:
            meta_title = ""
        out = []
        for page in reader.pages:
            try:
                out.append(page.extract_text() or "")
            except Exception:
                out.append("")
        return (meta_title or "").strip(), "\n".join(out).strip()
    except Exception:
        return "", ""

def scrape_pdf_local(pdf_path: Path) -> Tuple[str, str]:
    """Try pdfminer first; if thin, fall back to pypdf. Prefer metadata title."""
    text = extract_with_pdfminer(pdf_path)
    meta_title, alt_text = extract_with_pypdf(pdf_path)
    chosen_text = alt_text if len(alt_text.split()) > len(text.split()) else text
    title = meta_title or pdf_path.stem
    return title, (chosen_text or "").strip()

# -------------------------
# 4) Upload the PDF from your computer
# -------------------------
print("Please choose the local PDF to upload…")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")

uploaded_name = next(iter(uploaded.keys()))
src_path = Path("/content") / uploaded_name
pdf_path = Path("/content") / f"upload_{uuid.uuid4().hex}.pdf"
src_path.rename(pdf_path)
print(f"[OK] Uploaded & stored temporarily at: {pdf_path}")

# -------------------------
# 5) Normalize & Save to schema
# -------------------------
DOC_ID = "municipality_digital_data_charter"
JURISDICTION = "Municipal-Montreal"
SOURCE_TYPE = "Charter"
SOURCE_URL = "local-upload"  # uploaded file, not directly scraped from the Constellio URL

title, text = scrape_pdf_local(pdf_path)

row = {
    "doc_id": DOC_ID,
    "title": title,
    "jurisdiction": JURISDICTION,
    "source_type": SOURCE_TYPE,
    "url": SOURCE_URL,
    "retrieved_at_utc": now_utc_iso(),
    "language": safe_lang(text),
    "char_count": str(len(text)),
    "word_count": str(len(text.split())),
    "text": text,
}

warn_if_low_content(SOURCE_URL, row["title"], int(row["char_count"]), int(row["word_count"]))

out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
write_single_row_csv(out_path, row)
print(f"[OK] Saved per-link CSV: {out_path}")

append_to_master(MASTER_PATH, row)
print(f"[OK] Appended to master: {MASTER_PATH}")

# Cleanup temp file
try:
    pdf_path.unlink()
except Exception:
    pass

print("\nPreview:")
print(pd.read_csv(out_path, dtype=str).head(1).to_string(index=False))


Please choose the local PDF to upload…


Saving montreal_ged_92893_Digital Data Charter 2024 - Ville de Montréal.pdf to montreal_ged_92893_Digital Data Charter 2024 - Ville de Montréal.pdf
[OK] Uploaded & stored temporarily at: /content/upload_9da17abc2624417f91c52586208e8fd2.pdf
[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/municipality_digital_data_charter.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                           doc_id                                   title       jurisdiction source_type          url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                 

4. muncipality_open_data_policy  
https://donnees.montreal.ca/en/pages/our-approach name of file  

In [ ]:
# =========================================
# Municipality Link (uploaded PDF)
# File (CSV out): muncipality_open_data_policy.csv
# Strict schema + low-content warning
# =========================================


# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text
from pypdf import PdfReader

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only PDF, we can add OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) Robust PDF Text Extraction (from uploaded file)
# -------------------------
def extract_with_pdfminer(pdf_path: Path) -> str:
    try:
        return pdf_extract_text(str(pdf_path)) or ""
    except Exception:
        return ""

def extract_with_pypdf(pdf_path: Path) -> Tuple[str, str]:
    """Return (metadata_title, text) using pypdf (handles many encrypted PDFs)."""
    try:
        reader = PdfReader(str(pdf_path))
        if reader.is_encrypted:
            try:
                reader.decrypt("")  # empty password often works for public docs
            except Exception:
                pass
        meta_title = ""
        try:
            info = reader.metadata
            if info and getattr(info, "title", None):
                meta_title = str(info.title)
        except Exception:
            meta_title = ""
        out = []
        for page in reader.pages:
            try:
                out.append(page.extract_text() or "")
            except Exception:
                out.append("")
        return (meta_title or "").strip(), "\n".join(out).strip()
    except Exception:
        return "", ""

def scrape_pdf_local(pdf_path: Path) -> Tuple[str, str]:
    """Try pdfminer first; if thin, fall back to pypdf. Prefer metadata title."""
    text = extract_with_pdfminer(pdf_path)
    meta_title, alt_text = extract_with_pypdf(pdf_path)
    chosen_text = alt_text if len(alt_text.split()) > len(text.split()) else text
    title = meta_title or pdf_path.stem
    return title, (chosen_text or "").strip()

# -------------------------
# 4) Upload the PDF from your computer
# -------------------------
print("Please choose the local PDF to upload…")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")

uploaded_name = next(iter(uploaded.keys()))
src_path = Path("/content") / uploaded_name
pdf_path = Path("/content") / f"upload_{uuid.uuid4().hex}.pdf"
src_path.rename(pdf_path)
print(f"[OK] Uploaded & stored temporarily at: {pdf_path}")

# -------------------------
# 5) Normalize & Save to schema
# -------------------------
DOC_ID = "muncipality_open_data_policy"  # using your requested filename exactly
JURISDICTION = "Municipal-Montreal"
SOURCE_TYPE = "Policy"
SOURCE_URL = "local-upload"  # uploaded file (not fetched from the website link)

title, text = scrape_pdf_local(pdf_path)

row = {
    "doc_id": DOC_ID,
    "title": title,
    "jurisdiction": JURISDICTION,
    "source_type": SOURCE_TYPE,
    "url": SOURCE_URL,
    "retrieved_at_utc": now_utc_iso(),
    "language": safe_lang(text),
    "char_count": str(len(text)),
    "word_count": str(len(text.split())),
    "text": text,
}

warn_if_low_content(SOURCE_URL, row["title"], int(row["char_count"]), int(row["word_count"]))

out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
write_single_row_csv(out_path, row)
print(f"[OK] Saved per-link CSV: {out_path}")

append_to_master(MASTER_PATH, row)
print(f"[OK] Appended to master: {MASTER_PATH}")

# Cleanup temp file
try:
    pdf_path.unlink()
except Exception:
    pass

print("\nPreview:")
print(pd.read_csv(out_path, dtype=str).head(1).to_string(index=False))


Please choose the local PDF to upload…


Saving open_data_policy.pdf to open_data_policy.pdf
[OK] Uploaded & stored temporarily at: /content/upload_6fc315af19c54c7ebc880ef9641361f8.pdf
[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/muncipality_open_data_policy.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                      doc_id                                   title       jurisdiction source_type          url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

5. municipality_data_gov_directive
https://depot.ville.montreal.qc.ca/documents/data-governance-directive.pdf

In [ ]:
# =========================================
# Municipality Link (uploaded PDF)
# File (CSV out): municipality_data_gov_directive.csv
# Strict schema + low-content warning
# =========================================



# -------------------------
# 1) Paths & Imports
# -------------------------
from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/webscrape_links")
PER_LINK_DIR = BASE_DIR / "links"
MASTER_PATH = BASE_DIR / "master_links.csv"
PER_LINK_DIR.mkdir(parents=True, exist_ok=True)

import csv
import re
import uuid
from datetime import datetime, timezone
from typing import Dict, Any, Optional, Tuple

import pandas as pd
from langdetect import detect as lang_detect
from pdfminer.high_level import extract_text as pdf_extract_text
from pypdf import PdfReader

# -------------------------
# 2) Strict Schema & Utils
# -------------------------
SCHEMA = [
    "doc_id",
    "title",
    "jurisdiction",
    "source_type",
    "url",
    "retrieved_at_utc",
    "language",
    "char_count",
    "word_count",
    "text",
]

LOW_CHAR_THRESHOLD = 500
LOW_WORD_THRESHOLD = 100

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat()

def safe_lang(text: str) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    try:
        return lang_detect(text)
    except Exception:
        return ""

def count_chars_words(text: str) -> Tuple[int, int]:
    text = text or ""
    return len(text), len(text.split())

def warn_if_low_content(url: str, title: str, chars: int, words: int) -> None:
    if chars < LOW_CHAR_THRESHOLD or words < LOW_WORD_THRESHOLD:
        print(
            f"[WARNING] Low content\n"
            f"  url: {url}\n"
            f"  title: {title[:120]}\n"
            f"  char_count={chars} (<{LOW_CHAR_THRESHOLD}?)  "
            f"word_count={words} (<{LOW_WORD_THRESHOLD}?)\n"
            f"  Note: If this is a scanned/image-only PDF, we can add OCR later."
        )

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = ""
    return df[SCHEMA]

def write_single_row_csv(path: Path, row: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=SCHEMA)
        writer.writeheader()
        writer.writerow({k: row.get(k, "") for k in SCHEMA})

def append_to_master(master_path: Path, row: Dict[str, Any]) -> None:
    if master_path.exists():
        df = pd.read_csv(master_path, dtype=str, keep_default_na=False)
    else:
        df = pd.DataFrame(columns=SCHEMA)
    df = ensure_schema(df)
    df = pd.concat([df, pd.DataFrame([{k: str(row.get(k, "")) for k in SCHEMA}])], ignore_index=True)
    df = ensure_schema(df)
    df.to_csv(master_path, index=False, encoding="utf-8")

# -------------------------
# 3) Robust PDF Text Extraction (from uploaded file)
# -------------------------
def extract_with_pdfminer(pdf_path: Path) -> str:
    try:
        return pdf_extract_text(str(pdf_path)) or ""
    except Exception:
        return ""

def extract_with_pypdf(pdf_path: Path) -> Tuple[str, str]:
    """Return (metadata_title, text) using pypdf (handles many encrypted PDFs)."""
    try:
        reader = PdfReader(str(pdf_path))
        if reader.is_encrypted:
            try:
                reader.decrypt("")  # empty password often works for public docs
            except Exception:
                pass
        meta_title = ""
        try:
            info = reader.metadata
            if info and getattr(info, "title", None):
                meta_title = str(info.title)
        except Exception:
            meta_title = ""
        out = []
        for page in reader.pages:
            try:
                out.append(page.extract_text() or "")
            except Exception:
                out.append("")
        return (meta_title or "").strip(), "\n".join(out).strip()
    except Exception:
        return "", ""

def scrape_pdf_local(pdf_path: Path) -> Tuple[str, str]:
    """Try pdfminer first; if thin, fall back to pypdf. Prefer metadata title."""
    text = extract_with_pdfminer(pdf_path)
    meta_title, alt_text = extract_with_pypdf(pdf_path)
    chosen_text = alt_text if len(alt_text.split()) > len(text.split()) else text
    title = meta_title or pdf_path.stem
    return title, (chosen_text or "").strip()

# -------------------------
# 4) Upload the PDF from your computer
# -------------------------
print("Please choose the local PDF to upload…")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")

uploaded_name = next(iter(uploaded.keys()))
src_path = Path("/content") / uploaded_name
pdf_path = Path("/content") / f"upload_{uuid.uuid4().hex}.pdf"
src_path.rename(pdf_path)
print(f"[OK] Uploaded & stored temporarily at: {pdf_path}")

# -------------------------
# 5) Normalize & Save to schema
# -------------------------
DOC_ID = "municipality_data_gov_directive"
JURISDICTION = "Municipal-Montreal"
SOURCE_TYPE = "Directive"
SOURCE_URL = "local-upload"  # uploaded file (not fetched directly from the URL)

title, text = scrape_pdf_local(pdf_path)

row = {
    "doc_id": DOC_ID,
    "title": title,
    "jurisdiction": JURISDICTION,
    "source_type": SOURCE_TYPE,
    "url": SOURCE_URL,
    "retrieved_at_utc": now_utc_iso(),
    "language": safe_lang(text),
    "char_count": str(len(text)),
    "word_count": str(len(text.split())),
    "text": text,
}

warn_if_low_content(SOURCE_URL, row["title"], int(row["char_count"]), int(row["word_count"]))

out_path = PER_LINK_DIR / f"{DOC_ID}.csv"
write_single_row_csv(out_path, row)
print(f"[OK] Saved per-link CSV: {out_path}")

append_to_master(MASTER_PATH, row)
print(f"[OK] Appended to master: {MASTER_PATH}")

# Cleanup temp file
try:
    pdf_path.unlink()
except Exception:
    pass

print("\nPreview:")
print(pd.read_csv(out_path, dtype=str).head(1).to_string(index=False))


Please choose the local PDF to upload…


Saving data-governance-directive.pdf to data-governance-directive.pdf
[OK] Uploaded & stored temporarily at: /content/upload_ae3d1b0cdd4748698296c90174614abe.pdf
[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/municipality_data_gov_directive.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                         doc_id                     title       jurisdiction source_type          url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

6. municipality_gov_personal_info
https://montreal.ca/en/topics/governance-personal-information

In [ ]:
# =========================================
# Municipality Link: Governance of personal information (Ville de Montréal)
# File: municipality_gov_personal_info.csv
# =========================================

link_cfg = {
    "url": "https://montreal.ca/en/topics/governance-personal-information",
    "method": "html",
    "jurisdiction": "Municipal-Montreal",
    "source_type": "Directive",
    "doc_id": "municipality_gov_personal_info",
    "title_override": "Governance of personal information (Ville de Montréal)",
    # Robust extraction + CSS fallback for montreal.ca
    "css_title": "main h1, header h1, .page-title, h1",
    "css_text": "main, article, .content, #content",
    "use_trafilatura": True,
}

_ = process_link(6, link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "municipality_gov_personal_info.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/municipality_gov_personal_info.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                        doc_id                                                  title       jurisdiction source_type                                                           url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

7. municipality_confidential_policy
https://montreal.ca/en/topics/confidentiality-policy

In [ ]:
# =========================================
# Link 7: Confidentiality policy (Ville de Montréal)
# File: municipality_confidential_policy.csv
# =========================================

link_cfg = {
    "url": "https://montreal.ca/en/topics/confidentiality-policy",
    "method": "html",
    "jurisdiction": "Municipal-Montreal",
    "source_type": "Policy",
    "doc_id": "municipality_confidential_policy",
    "title_override": "Confidentiality policy (Ville de Montréal)",
    # Robust extraction + CSS fallback for montreal.ca pages
    "css_title": "main h1, header h1, .page-title, h1",
    "css_text": "main, article, .content, #content",
    "use_trafilatura": True,
}

_ = process_link(7, link_cfg)

print("\nPreview:")
print(pd.read_csv(PER_LINK_DIR / "municipality_confidential_policy.csv", dtype=str).head(1).to_string(index=False))

print(f"\nPer-link CSVs directory: {PER_LINK_DIR}")
print(f"Master CSV path:         {MASTER_PATH}")


[OK] Saved per-link CSV: /content/drive/MyDrive/webscrape_links/links/municipality_confidential_policy.csv
[OK] Appended to master: /content/drive/MyDrive/webscrape_links/master_links.csv

Preview:
                          doc_id                                      title       jurisdiction source_type                                                  url          retrieved_at_utc language char_count word_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [ ]:
# ============================================================
# Update Master File with Short Titles
# - Reads:  /content/drive/MyDrive/webscrape_links/master_links.csv
# - Adds:   short_title column
# - Saves:  /content/drive/MyDrive/webscrape_links/master_links_with_short_titles.csv
# ============================================================

from google.colab import drive
from pathlib import Path
import pandas as pd
import csv

# Mount Drive
drive.mount('/content/drive', force_remount=True)

BASE_DIR    = Path("/content/drive/MyDrive/webscrape_links")
INPUT_PATH  = BASE_DIR / "master_links.csv"
OUTPUT_PATH = BASE_DIR / "master_links_with_short_titles.csv"

# 1. Define the mapping
mapping = {
    'federal_guide_use_gen_ai': 'Guide on Gen AI',
    'federal_algorithmic_impact_assessment': 'AIA',
    'gc_ai_strategy_overview': 'AI Strategy',
    'federal_responsible_use_ai': 'Guiding Principles for AI',
    'federal_2023_26_data_strategy': 'Data Strategy',
    'policy_service_digital': 'Policy on Service and Digital',
    'federal_automated_decision_making': 'Directive on Automated Decision-Making',
    'provincial_strategie_integration_IA': 'Strategy for AI Integration',
    'provincial_good_practices_gen_AI': 'Gen AI Best Practices',
    'provincial_responsible_public_AI': 'AI Statement of Principles',
    'provincial_strat_cybersecurity_2024-28': 'Cybersecurity and Digital Strategy',
    'municipality_responsible_AI': 'Montreal Declaration',
    'municipality_strat_responsible_AI': 'AI Integration Strategy',
    'muncipality_open_data_policy': 'Open Data Policy',
    'municipality_data_gov_directive': 'Data Governance Directive',
    'municipality_digital_data_charter': 'Digital Data Charter',
    'municipality_gov_personal_info': 'PII Governance',
    'municipality_confidential_policy': 'Confidentiality Policy'
}

# 2. Load the original dataset
df = pd.read_csv(INPUT_PATH, dtype=str, keep_default_na=False, quoting=csv.QUOTE_ALL)

# 3. Apply the mapping to create the short_title column
df['short_title'] = df['doc_id'].map(mapping)

# 4. Save back to Google Drive
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL, lineterminator="\n")

print(f"Successfully saved updated file to: {OUTPUT_PATH}")